In [61]:
import dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
import os
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
import getpass
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
import sys
sys.tracebacklimit = 0
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [4]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
OPENAI_DEPLOYMENT = os.environ.get("OPENAI_DEPLOYMENT")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL")
EMBEDDING_DEPLOYMENT = os.environ.get("EMBEDDING_DEPLOYMENT")
OPENAI_MODEL_GPT4 = os.environ.get("OPENAI_MODEL_GPT4")
OPENAI_DEPLOYMENT_GPT4 = os.environ.get("OPENAI_DEPLOYMENT_GPT4")

In [5]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION)
llm = AzureChatOpenAI(model_name=OPENAI_MODEL, azure_deployment=OPENAI_DEPLOYMENT,temperature=0)
llm_gpt4 = AzureChatOpenAI(model_name=OPENAI_MODEL_GPT4, azure_deployment=OPENAI_DEPLOYMENT_GPT4,temperature=0)

In [19]:
def evaluation_llm(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False
    )
    return result

In [7]:
def evaluation_rag(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  # most likely
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False,
    )
    return result

In [8]:
questions = [
    "Finish writing the following short report: On February 24, 2024, former President Donald Trump secured a significant victory in the South Carolina Republican primary, defeating Nikki Haley in her home state. This report aims to analyze the key factors leading to Trump's success, the impact on the ongoing presidential race, and potential challenges for other candidates.",
    "Finish writing the following short report: Amidst the mysterious death of Russian opposition leader Alexei Navalny on February 16, 2024, this report investigates the claims made by his colleague, Maria Pevchikh, suggesting that Navalny was on the brink of being freed in a prisoner swap. The alleged plan involved exchanging Navalny for Russian hitman Vadim Krasikov, serving a life sentence in Germany, and two US citizens held in Russia. The report delves into the events leading up to Navalny's demise, the proposed prisoner swap, and the geopolitical implications surrounding this tragic incident.",
    "Please provide a concise report on the Israel-Hamas conflict, focusing on the recent Israeli special forces operation at Nasser Hospital in Khan Younis, Gaza. Include information on the reasons for the operation, international reactions, and the current state of the conflict.",
    "Finish writing the following report: In a recent incident involving an Alaska Airlines Boeing 737 Max 9, a door blowout occurred shortly after take-off, prompting the grounding of 171 planes of the same model in the United States. Boeing's president and CEO, Dave Calhoun, publicly admitted the company's mistake, acknowledging that a missing 27kg door 'plug' was not properly secured.",
    "Finish writing the following short report: Elon Musk's desire for increased control over Tesla has sparked discussions about his stake and concerns regarding potential takeovers.",
    "Finish writing the following short report: Hollywood remains unfazed by OpenAI's Sora, the latest text-to-video generator, despite its ability to quickly produce high-quality videos from simple prompts.",
    "Finish writing the following short report: Prince William's unexpected withdrawal from the memorial service for the late former King Constantine of Greece.",
    "Finish writing the following short report: As Ukraine commemorates the second anniversary of Russia's invasion, Europe confronts pressing questions about sustaining financial support for the war-torn nation."
]

ground_truths = [
    ["Donald Trump won the South Carolina Republican primary on February 24, 2024. The victory solidifies Trump's path towards securing the GOP nomination for the third consecutive time. Nikki Haley, Trump's former representative to the UN and former South Carolina governor, faced challenges despite holding more campaign events. South Carolina's first-in-the-South primary has historically been a reliable indicator of the eventual Republican nominee. Trump has won all significant contests for Republican delegates, including Iowa, New Hampshire, Nevada, and the US Virgin Islands. Haley, despite vowing to continue running, faces pressure to reconsider her candidacy after Trump's South Carolina victory. Trump's strength endures despite facing 91 criminal charges related to efforts to overturn the 2020 election and other controversies. Trump's first criminal trial is scheduled for March 25, 2024, in New York. The 2024 presidential race is increasingly shaping up as a rematch between Trump and President Joe Biden. Biden, facing criticism for supporting Israel in the Gaza War, is preparing for a tough electoral battle against Trump.Trump's age (77) and Biden's age (81) become focal points in the election discourse, with both candidates trading jabs."],
    ["Navalny died on February 16, 2024, in a Siberian prison colony while serving a 19-year sentence. Maria Pevchikh claimed that Navalny was about to be freed in a prisoner swap involving Vadim Krasikov and two US citizens. Kremlin spokesperson Dmitry Peskov denied knowledge of any such agreements or negotiations. Yulia Navalnaya accused Vladimir Putin of killing her husband and demanded the return of Navalny's body. Navalny's body was reportedly held for 'chemical analysis,' leading to suspicions of poisoning with Novichok. International leaders, including the EU and the US, held Putin responsible for Navalny's death and considered potential sanctions. Navalny's opposition activities, his previous poisoning in 2020, and the role of economic policies, particularly mass privatization, in shaping the conditions leading to his tragic death."],
    ["Israeli special forces entered Nasser Hospital in Khan Younis, Gaza, based on 'credible intelligence' that Hamas held captives from the October 7 attacks. The operation aimed to locate bodies of Israeli hostages. Prime Minister Netanyahu vowed to proceed with an invasion of Rafah. Prince William expressed concern over the conflict, emphasizing the need for humanitarian support, hostage release, and permanent peace. The report should cover these key points, incorporating relevant details and developments."],
    ["""Alaska Airlines Boeing 737 Max 9 door blowout incident.
Grounding of 171 Boeing 737 Max 9 planes in the U.S.
Boeing's acknowledgment of the mistake by Dave Calhoun.
Details about the missing 27kg door "plug" and the four crucial bolts.
Rapid loss of cabin pressure and the National Transportation Safety Board's preliminary findings.
Highlight of manufacturing defects and quality control issues at Boeing.
Mention of executive changes at Boeing.
The strain on Boeing's relationship with airlines and increased safety concerns. The missing section of the plane was retrieved from the back garden of a Portland teacher.
Alaska Airlines had placed restrictions on the aircraft following pressurization warnings before the incident.
Boeing's president and CEO, Dave Calhoun, reassured Boeing staff, promising 100% transparency in addressing the mistake.
Boeing organized a "Quality Stand Down" at its 737 factory in Renton, Washington, focusing on quality issues.
The National Safety Transportation Board (NTSB) is expected to release its preliminary findings on the Alaska Airlines 737 Max 9 incident.
Critics argue that Boeing's shift in corporate culture towards prioritizing profits over safety started after the 1997 merger with McDonnell Douglas.
Boeing's move away from its Pacific Northwest roots, the shift in leadership with financial backgrounds, and its reliance on suppliers are highlighted.
The report emphasizes the impact of cost-cutting measures on Boeing's engineering and manufacturing, leading to various safety and quality issues.
Boeing's financial struggles, including cumulative net losses of more than $26 billion over the last five years, are outlined.
A new FAA report finds major issues with Boeing's safety culture, describing it as "inadequate" and "confusing."
The report identifies gaps in Boeing's safety journey, highlighting employee confusion, inconsistent reporting channels, and a fear of retaliation.
Boeing's safety improvements since 2019, such as establishing an aerospace safety committee and a chief aerospace safety office, are mentioned."""],
    ["Elon Musk's desire for increased control over Tesla has sparked discussions about his stake and concerns regarding potential takeovers. Elon Musk, with approximately 13% ownership in Tesla, expressed a desire for increased control due to concerns about potential takeovers, particularly by 'dubious interests.' He highlighted unease regarding Tesla's investments in artificial intelligence and stated a preference for a 25% voting control. The comments led to a 'firestorm' for Tesla, impacting its self-driving and automated capabilities narrative. Additionally, ongoing legal action surrounds a 10-year pay package granted to Musk in 2018, valued at about $55 billion."],
    ["Hollywood remains unfazed by OpenAI's Sora, the latest text-to-video generator, despite its ability to quickly produce high-quality videos from simple prompts. Industry experts express concerns about potential job displacement and creative challenges, with some figures like Tyler Perry already taking action. Sora, though not broadly available yet, has garnered attention for its impressive capabilities, generating hyperrealistic videos up to 1 minute long. Sora is the latest leap in artificial intelligence software, allowing users to create staggeringly realistic videos from a simple worded prompt"],
    ["Prince William's unexpected withdrawal from the memorial service for the late former King Constantine of Greece was attrubyted to a 'personal matter'.  In the absence of Prince William, Queen Camilla led the Royal Family at St George's Chapel in Windsor Castle. Despite concerns, royal sources assured the public that there was no cause for alarm. "],
    ["As Ukraine commemorates the second anniversary of Russia's invasion, Europe confronts pressing questions about sustaining financial support for the war-torn nation. The war, deadlocked for some time, witnessed Ukraine's recent withdrawal from the key town of Avdiivka, marking its worst defeat since May. Financial aid from the United States, currently awaiting House approval, is crucial, with the EU and NATO unity showing signs of strain. Despite spending over $100 billion on Ukraine's defense, Europe is grappling with the challenge of political fatigue and the potential impact of upcoming elections. "]
    ]

In [46]:
max_average = 0

In [48]:
def find_highest(average_score):
    global max_average
    if average_score > max_average:
        max_average = average_score
        print("This is the new best value!")

In [44]:
def dictionary(result):
    dict_result = dict(result)
    average_score = sum(dict_result.values()) / len(dict_result)
    print(f"The average score is: {average_score}")
    find_highest(average_score)
    return average_score, dict_result

### General answer by LLMs

In [10]:
template = """{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
llm_chain =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm}
)
llm_chain_gpt4 =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm_gpt4}
)

In [15]:
answers_llm = []
contexts_llm = [[""],[""],[""],[""],[""],[""],[""],[""]]

In [17]:
for query in questions:
    response = llm_chain.invoke({"question": query})
    answers_llm.append(response["response"].content)

In [20]:
llm_results = evaluation_llm(questions, answers_llm, contexts_llm, ground_truths)
print(llm_results)
avg_llm_results ,dict_llm_results = dictionary(llm_results)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating:  19%|█▉        | 6/32 [00:01<00:06,  3.89it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 32/32 [00:07<00:00,  4.07it/s]


{'faithfulness': 0.9476, 'answer_relevancy': 0.5959, 'answer_similarity': 0.9552, 'answer_correctness': 0.6040}
The average score is: 0.7756666768868822


In [49]:
avg_llm_results ,dict_llm_results = dictionary(llm_results)

The average score is: 0.7756666768868822
This is the new best value!


In [21]:
answers_llm_gpt4 = []
for query in questions:
    response = llm_chain_gpt4.invoke({"question": query})
    answers_llm_gpt4.append(response["response"].content)
llm_results_gpt4 = evaluation_llm(questions, answers_llm_gpt4, contexts_llm, ground_truths)
print(llm_results_gpt4)
avg_llm_results_gpt4 ,dict_llm_results_gpt4 = dictionary(llm_results_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   3%|▎         | 1/32 [00:01<00:58,  1.90s/it]Runner in Executor raised an exception
TypeError: expected string or buffer
Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating:  22%|██▏       | 7/32 [00:02<00:05,  4.43it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 32/32 [00:08<00:00,  3.69it/s]


{'faithfulness': 0.6604, 'answer_relevancy': 0.8229, 'answer_similarity': 0.6442, 'answer_correctness': 0.8763}
The average score is: 0.750970840509609


In [50]:
avg_llm_results_gpt4 ,dict_llm_results_gpt4 = dictionary(llm_results_gpt4)

The average score is: 0.750970840509609


# RAG

### Naive RAG

In [22]:
def retrieval_chain(prompt, retriever, llm):
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
    )
    return retrieval_augmented_qa_chain

In [23]:
loader = DirectoryLoader('../news', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db_naive = Chroma.from_documents(chunks, embeddings_client, persist_directory = "../news/vectordb/naive")
db_naive.persist()
retriever_naive = db_naive.as_retriever()

In [24]:
db_naive = Chroma(persist_directory = "../news/vectordb/naive", embedding_function=embeddings_client)
retriever_naive = db_naive.as_retriever()

In [25]:
template = """{question}. 
Provided context {context}."""
prompt = ChatPromptTemplate.from_template(template)

In [26]:
answers_naive = []
contexts_naive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_naive, llm).invoke({"question": query})
        # Access the response content
        answers_naive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_naive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_naive.append("No answer")
        context_full = retriever_naive.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_naive.append(context_content)

In [27]:
result_naive_rag = evaluation_rag(questions, answers_naive, contexts_naive, ground_truths)
print(result_naive_rag)
avg_result_naive_rag, dict_result_naive_rag = dictionary(result_naive_rag)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:16<00:00,  2.86it/s]


{'faithfulness': 0.9292, 'answer_relevancy': 0.7477, 'context_precision': 0.9792, 'context_recall': 0.9259, 'answer_similarity': 0.9536, 'answer_correctness': 0.6087}
The average score is: 0.8573756052250902


In [51]:
avg_result_naive_rag, dict_result_naive_rag = dictionary(result_naive_rag)

The average score is: 0.8573756052250902
This is the new best value!


### recursive splitter

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder()
chunks_r = text_splitter.split_documents(documents)
db_basic = Chroma.from_documents(chunks_r, embeddings_client, persist_directory = "../news/vectordb/recursive_basic")
db_basic.persist()
retriever_basic = db_basic.as_retriever()

In [28]:
db_basic = Chroma(persist_directory = "../news/vectordb/recursive_basic", embedding_function=embeddings_client)
retriever_basic = db_basic.as_retriever()

In [29]:
answers_recursive = []
contexts_recursive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_basic, llm).invoke({"question": query})
        # Access the response content
        answers_recursive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_recursive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_recursive.append("No answer")
        context_full = retriever_basic.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_recursive.append(context_content)

In [30]:
result_recursive = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
print(result_recursive)
avg_result_recursive, dict_result_recursive = dictionary(result_recursive)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:20<00:00,  2.30it/s]


{'faithfulness': 1.0000, 'answer_relevancy': 0.8062, 'context_precision': 1.0000, 'context_recall': 0.8286, 'answer_similarity': 0.9233, 'answer_correctness': 0.6058}
The average score is: 0.8606532019220987


In [52]:
avg_result_recursive, dict_result_recursive = dictionary(result_recursive)

The average score is: 0.8606532019220987
This is the new best value!


### chunk sizes

In [33]:
def run_and_evaluate(retriever, prompt, llm):
    answers_recursive = []
    contexts_recursive = []

    for query in questions:
        try:  
            response = retrieval_chain(prompt, retriever, llm).invoke({"question": query})
            # Access the response content
            answers_recursive.append(response["response"].content)
            # Access the context content
            context_content = [context.page_content for context in response["context"]]
            contexts_recursive.append(context_content)  
        except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_recursive.append("No answer")
            context_full = retriever.get_relevant_documents(query)
            context_content = [context.page_content for context in context_full]
            contexts_recursive.append(context_content)


    result = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
    return result

chunk = 1000

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
chunks_1000 = text_splitter.split_documents(documents)
print(len(chunks_1000))
db_1000 = Chroma.from_documents(chunks_1000, embeddings_client, persist_directory = "../news/vectordb/recursive_1000")
db_1000.persist()

In [31]:
db_1000 = Chroma(persist_directory = "../news/vectordb/recursive_1000", embedding_function=embeddings_client)

In [34]:
retriever_1000 = db_1000.as_retriever()
result_1000 = run_and_evaluate(retriever_1000, prompt, llm)
print("CHUNK SIZE 1000")
print(result_1000)
avg_result_1000, dict_result_1000 = dictionary(result_1000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:16<00:00,  2.97it/s]


CHUNK SIZE 1000
{'faithfulness': 0.9750, 'answer_relevancy': 0.7460, 'context_precision': 0.9583, 'context_recall': 0.9009, 'answer_similarity': 0.9595, 'answer_correctness': 0.6084}
The average score is: 0.8580138744596889


In [53]:
avg_result_1000, dict_result_1000 = dictionary(result_1000)

The average score is: 0.8580138744596889


chunk = 500

In [33]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500, chunk_overlap = 50)
chunks_500 = text_splitter.split_documents(documents)
print(len(chunks_500))
db_500 = Chroma.from_documents(chunks_500, embeddings_client, persist_directory = "../news/vectordb/recursive_500")
db_500.persist()

107


In [35]:
db_500 = Chroma(persist_directory = "../news/vectordb/recursive_500", embedding_function=embeddings_client)

In [36]:
retriever_500 = db_500.as_retriever()
result_500 = run_and_evaluate(retriever_500, prompt, llm)
print("CHUNK SIZE 500")
print(result_500)
avg_result_500, dict_result_500 = dictionary(result_500)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:16<00:00,  2.92it/s]


CHUNK SIZE 500
{'faithfulness': 0.9476, 'answer_relevancy': 0.7588, 'context_precision': 0.9653, 'context_recall': 0.8492, 'answer_similarity': 0.9608, 'answer_correctness': 0.6713}
The average score is: 0.8588243339324139


In [54]:
avg_result_500, dict_result_500 = dictionary(result_500)

The average score is: 0.8588243339324139


chunk = 2000

In [35]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 2000, chunk_overlap = 200)
chunks_2000 = text_splitter.split_documents(documents)
print(len(chunks_2000))
db_2000 = Chroma.from_documents(chunks_2000, embeddings_client, persist_directory = "../news/vectordb/recursive_2000")


37


In [37]:
db_2000 = Chroma(persist_directory = "../news/vectordb/recursive_2000", embedding_function=embeddings_client)

In [38]:
retriever_2000 = db_2000.as_retriever()
result_2000 = run_and_evaluate(retriever_2000, prompt, llm)
print("CHUNK SIZE 2000")
print(result_2000)
avg_result_2000, dict_result_2000 = dictionary(result_2000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:16<00:00,  2.85it/s]


CHUNK SIZE 2000
{'faithfulness': 0.9298, 'answer_relevancy': 0.7010, 'context_precision': 0.9896, 'context_recall': 0.9375, 'answer_similarity': 0.9615, 'answer_correctness': 0.5258}
The average score is: 0.8408431168806643


In [55]:
avg_result_2000, dict_result_2000 = dictionary(result_2000)

The average score is: 0.8408431168806643


chunk = 3000

In [38]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 3000, chunk_overlap = 300)
chunks_3000 = text_splitter.split_documents(documents)
print(len(chunks_3000))
db_3000 = Chroma.from_documents(chunks_3000, embeddings_client, persist_directory = "../news/vectordb/recursive_3000")

32


In [39]:
db_3000 = Chroma(persist_directory = "../news/vectordb/recursive_3000", embedding_function=embeddings_client)

In [40]:
retriever_3000 = db_3000.as_retriever()
result_3000 = run_and_evaluate(retriever_3000, prompt, llm)
print("CHUNK SIZE 3000")
print(result_3000)
avg_result_3000 ,dict_result_3000 = dictionary(result_3000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:22<00:00,  2.11it/s]


CHUNK SIZE 3000
{'faithfulness': 0.9429, 'answer_relevancy': 0.8032, 'context_precision': 0.9896, 'context_recall': 0.9643, 'answer_similarity': 0.9165, 'answer_correctness': 0.6267}
The average score is: 0.8738612558505032


In [56]:
avg_result_3000 ,dict_result_3000 = dictionary(result_3000)

The average score is: 0.8738612558505032
This is the new best value!


### now time to look for different top-k

Note: We continue with the size chunk of 3000 as it had the highest average score

In [57]:
retriever_2 = db_3000.as_retriever(search_kwargs={"k": 2})
result_2 = run_and_evaluate(retriever_2, prompt, llm)
print("CHUNK SIZE 1000, K=2")
print(result_2)
avg_result_2, dict_result_2 = dictionary(result_2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:14<00:00,  3.29it/s]


CHUNK SIZE 1000, K=2
{'faithfulness': 0.9667, 'answer_relevancy': 0.7627, 'context_precision': 1.0000, 'context_recall': 0.8884, 'answer_similarity': 0.9599, 'answer_correctness': 0.6383}
The average score is: 0.8693258385468209


In [58]:
retriever_3 = db_3000.as_retriever(search_kwargs={"k": 3})
result_3 = run_and_evaluate(retriever_3, prompt, llm)
print("CHUNK SIZE 1000, K=3")
print(result_3)
avg_result_3, dict_result_3 = dictionary(result_3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:17<00:00,  2.70it/s]


CHUNK SIZE 1000, K=3
{'faithfulness': 0.9435, 'answer_relevancy': 0.7420, 'context_precision': 1.0000, 'context_recall': 0.9931, 'answer_similarity': 0.9548, 'answer_correctness': 0.6497}
The average score is: 0.8805190156333335
This is the new best value!


In [60]:
retriever_5 = db_3000.as_retriever(search_kwargs={"k": 5})
result_5 = run_and_evaluate(retriever_5, prompt, llm)
print("CHUNK SIZE 1000, K=5")
print(result_5)
avg_result_5, dict_result_5 = dictionary(result_5)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8369 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Evaluating:   4%|▍         | 2/48 [00:02<00:52,  1.14s/it]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8510 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_ex

CHUNK SIZE 1000, K=5
{'faithfulness': 0.7877, 'answer_relevancy': 0.6899, 'context_precision': 0.9040, 'context_recall': 0.9881, 'answer_similarity': 0.9665, 'answer_correctness': 0.6658}
The average score is: 0.8336666858719243


### look for different retrievers

3 chunks was the best score

#### parent document retriever

In [62]:
parent_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

# vectorstore = Chroma(collection_name="split_parents",persist_directory = "../news/vectordb/parent", embedding_function=embeddings_client)
# vectorstore.persist()
vectorstore = Chroma(persist_directory = "../news/vectordb/parent", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={"k": 3}
)
parent_document_retriever.add_documents(documents)
result_parent = run_and_evaluate(parent_document_retriever, prompt, llm)
print(result_parent)
avg_result_parent, dict_result_parent = dictionary(result_parent)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:14<00:00,  3.22it/s]


{'faithfulness': 1.0000, 'answer_relevancy': 0.6608, 'context_precision': 1.0000, 'context_recall': 0.9021, 'answer_similarity': 0.9569, 'answer_correctness': 0.6213}
The average score is: 0.8568481431776626


In [63]:
parent_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap = 50)
child_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap = 0)

# vectorstore = Chroma(collection_name="split_parents_small",persist_directory = "../news/vectordb/parent_small", embedding_function=embeddings_client)
# vectorstore.persist()
vectorstore = Chroma(persist_directory = "../news/vectordb/parent_small", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_small = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_small,
    parent_splitter=parent_splitter_small,
)
parent_document_retriever_small.add_documents(documents)
result_parent_small = run_and_evaluate(parent_document_retriever_small, prompt, llm)
print(result_parent_small)
avg_result_parent_small, dict_result_parent_small = dictionary(result_parent_small)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:13<00:00,  3.66it/s]


{'faithfulness': 0.9524, 'answer_relevancy': 0.6514, 'context_precision': 1.0000, 'context_recall': 0.7962, 'answer_similarity': 0.9561, 'answer_correctness': 0.6182}
The average score is: 0.8290381152615773


In [64]:
parent_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=3000)
child_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap = 0)

# vectorstore = Chroma(collection_name="split_parents_large",persist_directory = "../news/vectordb/parent_large", embedding_function=embeddings_client)
# vectorstore.persist()
vectorstore = Chroma(persist_directory = "../news/vectordb/parent_large", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_large = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_large,
    parent_splitter=parent_splitter_large,
)
parent_document_retriever_large.add_documents(documents)
result_parent_large = run_and_evaluate(parent_document_retriever_large, prompt, llm)
print(result_parent_large)
avg_result_parent_large, dict_result_parent_large = dictionary(result_parent_large)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:17<00:00,  2.75it/s]


{'faithfulness': 0.9667, 'answer_relevancy': 0.7585, 'context_precision': 1.0000, 'context_recall': 0.9271, 'answer_similarity': 0.9559, 'answer_correctness': 0.6066}
The average score is: 0.869122662537911


#### Maximum marginal relevance retrieval

In [65]:
retriever_mmr = db_3000.as_retriever(search_type="mmr",search_kwargs={"k": 3})
result_mmr = run_and_evaluate(retriever_mmr, prompt, llm)
print("Marginal relevance")
print(result_mmr)
average_score_mmr, dict_result_mmr = dictionary(result_mmr)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:13<00:00,  3.53it/s]


Marginal relevance
{'faithfulness': 0.9323, 'answer_relevancy': 0.7449, 'context_precision': 1.0000, 'context_recall': 0.8599, 'answer_similarity': 0.9202, 'answer_correctness': 0.6089}
The average score is: 0.8443525883325994


#### BM25

In [67]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 3000, chunk_overlap = 300)
chunks_3000 = text_splitter.split_documents(documents)

In [68]:
retriever_bm25 = BM25Retriever.from_documents(chunks_3000, search_kwargs = {"k": 3})
result_bm25 = run_and_evaluate(retriever_bm25, prompt, llm)
print("BM25")
print(result_bm25)
average_score_bm25, dict_result_bm25 = dictionary(result_bm25)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:18<00:00,  2.63it/s]


BM25
{'faithfulness': 0.9476, 'answer_relevancy': 0.8068, 'context_precision': 0.9687, 'context_recall': 0.6786, 'answer_similarity': 0.9463, 'answer_correctness': 0.6209}
The average score is: 0.828169206089286


#### Ensambler - Hybrid

In [69]:
ensemble_retriever_1 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.75, 0.25])
result_ensemble1 = run_and_evaluate(ensemble_retriever_1, prompt, llm)
print("Ensambler 75/25")
print(result_ensemble1)
average_score_ensambler1, dict_result_ensemble1 = dictionary(result_ensemble1)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  98%|█████████▊| 47/48 [00:18<00:02,  2.06s/it]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8347 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Evaluating: 100%|██████████| 48/48 [03:16<00:00,  4.10s/it]


Ensambler 75/25
{'faithfulness': 0.7398, 'answer_relevancy': 0.7754, 'context_precision': 0.8956, 'context_recall': 0.8333, 'answer_similarity': 0.9252, 'answer_correctness': 0.8056}
The average score is: 0.8291552121489785


In [70]:
ensemble_retriever_2 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.5, 0.5])
result_ensemble2 = run_and_evaluate(ensemble_retriever_2, prompt, llm)
print("Ensambler 50/50")
print(result_ensemble2)
avg_result_ensemble2, dict_result_ensemble2 = dictionary(result_ensemble2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  98%|█████████▊| 47/48 [00:17<00:02,  2.15s/it]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8347 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Evaluating: 100%|██████████| 48/48 [02:22<00:00,  2.97s/it]


Ensambler 50/50
{'faithfulness': 0.7592, 'answer_relevancy': 0.8460, 'context_precision': 0.8933, 'context_recall': 0.7396, 'answer_similarity': 0.9268, 'answer_correctness': 0.6889}
The average score is: 0.8089544473795994


In [71]:
ensemble_retriever_3 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.25,0.75])
result_ensemble3 = run_and_evaluate(ensemble_retriever_3, prompt, llm)
print("Ensambler 25/75")
print(result_ensemble3)
avg_result_ensemble3, dict_result_ensemble3 = dictionary(result_ensemble3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  98%|█████████▊| 47/48 [00:16<00:01,  1.80s/it]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8347 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Evaluating: 100%|██████████| 48/48 [02:03<00:00,  2.58s/it]


Ensambler 25/75
{'faithfulness': 0.7341, 'answer_relevancy': 0.8545, 'context_precision': 0.9128, 'context_recall': 0.7396, 'answer_similarity': 0.9336, 'answer_correctness': 0.7116}
The average score is: 0.8143688626800305


#### Multi-stage - reranker

In [72]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [73]:
retriever_context = retriever_3
compressor = CohereRerank(top_n = 3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context
)

result_compression = run_and_evaluate(compression_retriever, prompt, llm)
print("Reranker")
print(result_compression)
avg_result_compression, dict_result_compression = dictionary(result_compression)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:18<00:00,  2.66it/s]


Reranker
{'faithfulness': 0.9009, 'answer_relevancy': 0.8555, 'context_precision': 1.0000, 'context_recall': 0.9443, 'answer_similarity': 0.9532, 'answer_correctness': 0.6482}
The average score is: 0.8836811674501047
This is the new best value!


#### creating context by remaking the query

In [74]:
template_context = "Generate a search query to fetch the relevant documents using the user's {question}. Craft a query that specifically targets the keywords in the question. In the answer provide only the query."
prompt_context = ChatPromptTemplate.from_template(template_context)

In [75]:
answers_final = []
contexts_final = []
llm_for_context =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt_context | llm}
)
for query in questions:
    response_check = llm_for_context.invoke({"question": query})
    search_query = response_check["response"].content
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("context"), "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
)
    docs = compression_retriever.get_relevant_documents(search_query)
    formatted_docs = []
    for doc in docs:
        resulting_doc = doc.page_content
        formatted_docs.append(resulting_doc)
    try:  
            response = retrieval_augmented_qa_chain.invoke({"context": formatted_docs, "question": query})
            # Access the response content
            answers_final.append(response["response"].content)
            contexts_final.append(formatted_docs)  
    except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_final.append("No answer")
            contexts_final.append(formatted_docs)


result_search_query = evaluation_rag(questions, answers_final, contexts_final, ground_truths)
print(result_search_query)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:20<00:00,  2.33it/s]


{'faithfulness': 0.9476, 'answer_relevancy': 0.7604, 'context_precision': 0.9792, 'context_recall': 0.9368, 'answer_similarity': 0.9580, 'answer_correctness': 0.6228}


In [76]:
avg_result_search_query, dict_result_search_query = dictionary(result_search_query)

The average score is: 0.8674580208321929


### change model to GPT-4

In [77]:
compressor = CohereRerank(top_n = 3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context
)

result_compression_gpt4 = run_and_evaluate(compression_retriever, prompt, llm_gpt4)
print("Reranker")
print(result_compression_gpt4)
avg_result_compression_gpt4, dict_result_compression_gpt4 = dictionary(result_compression_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 48/48 [00:17<00:00,  2.76it/s]


Reranker
{'faithfulness': 0.9796, 'answer_relevancy': 0.7408, 'context_precision': 1.0000, 'context_recall': 0.9752, 'answer_similarity': 0.9484, 'answer_correctness': 0.6897}
The average score is: 0.8889578908969379
This is the new best value!
